In [1]:
import utils.evaluation as eval_utils
import utils.model as model_utils
import torch
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils.embed_concepts as embed_concepts
import tqdm.auto as tqdm
import os
import pickle

In [3]:
FILE_CONFIG = {
    "path_all_concept": "D:/lora_finetune_eval/basic_info/concept_all.parquet",
    "path_is_a" : "D:/lora_finetune_eval/basic_info/graph_is_a_invariant.csv",
    "mapped_concept": "D:/lora_finetune_eval/basic_info/mapped_concepts_2025-04-01.csv",
    "training_triplet_idx": "D:/lora_finetune_eval/basic_info/training_anchor_idx_1M.parquet",

    "icd_snomed" : "D:/lora_finetune_eval/icd_snomed/",

    "embedding_save_path": "D:/lora_finetune_eval/embedding_by_model/",
    "syn_embedding_save_path": "D:/lora_finetune_eval/syn_embedding_by_model/",
    "new_exp_embedding_save_path": "D:/lora_finetune_eval/new_exp_embedding_by_model/",
    "embedding_icd_snomed_save_path": "D:/lora_finetune_eval/embedding_icd_snomed_by_model/",

    "result_save_path": "D:/lora_finetune_eval/result_by_model/",

}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model_names = embed_concepts.MODEL_CONFIG.keys()

df_concept_all = pl.read_parquet(FILE_CONFIG["path_all_concept"])
df_concept_all_idx = set(df_concept_all["idx"].unique().to_list())

df_concept_train = pl.read_parquet(FILE_CONFIG["training_triplet_idx"])
df_concept_train_idx = set(df_concept_train["idx"].unique().to_list())

df_concept_test_idx = df_concept_all_idx - df_concept_train_idx
df_concept_test = list(df_concept_test_idx)

id2idx = dict(zip(df_concept_all["id"], df_concept_all["idx"]))

df_concept_test_fd = df_concept_all.filter(pl.col("idx").is_in(df_concept_test)).filter(pl.col("status") == "defined")["idx"].unique().to_list()


# task 1: retrieve label given expression

In [ ]:
mrrs_1 = []
models_1 = []
ranks_1 = {}
for model_key in model_names:

    if os.path.exists(FILE_CONFIG["embedding_save_path"] + model_key + ".pt"):
        print(f"Evaluating {model_key}...")
        embeddings = torch.load(FILE_CONFIG["embedding_save_path"] + model_key + ".pt")
        embedding_exp= embeddings["expressions_embeddings"]
        embedding_label = embeddings["labels_embeddings"]
        rank = eval_utils.top_k_array_by_batch(df_concept_test_fd, embedding_exp, embedding_label,device, 100)
        mrr_rank = eval_utils.compute_mmr(rank)
        mrrs_1.append(mrr_rank)
        models_1.append(model_key)
        ranks_1[model_key] = rank
        print(f"MRR: {mrr_rank}")

    else: 
        continue
    


In [18]:
pl.DataFrame({"model": models_1, "mrr_1": mrrs_1}).write_csv(FILE_CONFIG["result_save_path"] + "mrr_1.csv")
with open(FILE_CONFIG["result_save_path"] + "ranks_1.pkl", 'wb') as f:
    pickle.dump(ranks_1, f)



# task 2,3

In [5]:
mrrs_2_exp = []
mrrs_2_label = []

models_2 = []
ranks_2 = {}
for model_key in model_names:

    if  os.path.exists(FILE_CONFIG["syn_embedding_save_path"] + model_key + ".pt"):
        print(f"Evaluating {model_key}...")

        syn_embeddings = torch.load(FILE_CONFIG["syn_embedding_save_path"] + model_key + ".pt")
        embeddings = torch.load(FILE_CONFIG["embedding_save_path"] + model_key + ".pt")

        embedding_exp = embeddings["expressions_embeddings"]
        embedding_label = embeddings["labels_embeddings"]


        embedding_syn_idx = syn_embeddings["idx"]
        embedding_syn_matrix = syn_embeddings["expressions_embeddings"]

        # Determine test indices
        embedding_syn_test_idx = list(set(embedding_syn_idx).intersection(set(df_concept_test_fd)))

        # Compute rankings
        rank_w_exp = eval_utils.top_k_array_syn(embedding_syn_test_idx, embedding_syn_idx, embedding_syn_matrix, embedding_exp, device=device)
        rank_w_label = eval_utils.top_k_array_syn(embedding_syn_test_idx, embedding_syn_idx, embedding_syn_matrix, embedding_label, device=device)
        
        mrr_rank_exp = eval_utils.compute_mmr(rank_w_exp)
        mrr_rank_label = eval_utils.compute_mmr(rank_w_label)


        mrrs_2_exp.append(mrr_rank_exp)
        mrrs_2_label.append(mrr_rank_label)

        models_2.append(model_key)

        ranks_2[model_key]['expession'] = rank_w_exp
        ranks_2[model_key]['label'] = rank_w_label


        print(f"MRR: {mrr_rank_exp}")
        print(f"MRR: {mrr_rank_label}")

    else: 
        continue


Evaluating finetune_1M...


c:\Users\yy\Desktop\quantizer\vector-quantize-pytorch\utils\evaluation.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_matrix = torch.tensor(query_matrix, dtype=torch.float32).to(device)
c:\Users\yy\Desktop\quantizer\vector-quantize-pytorch\utils\evaluation.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  candidate_matrix = torch.tensor(candidate_matrix, dtype=torch.float32).to(device)


0 / 183000


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x768 and 384x392641)

In [35]:
mrrs_2_exp

[]

#